In [1]:
########## SET-UP ##########

# necessary libraries
import os
import json
import requests
import time
import datetime

# INSERT YOUR API KEY HERE
KEY = 'b00c992b-6fc1-4795-abcc-7db85c1b72fa'


In [2]:
# MATCH-v2.2
REGION_ENDPOINT = "https://{0}.api.pvp.net/api/lol/{0}/"

# FUNCTION TO PULL MATCH DATA
def get_match(REGION, matchId, includeTimeline):
    """
    Retrieve match by match ID.
    """
    return requests.get(
        (REGION_ENDPOINT + "v2.2/match/{1}?"
         "api_key={2}&includeTimeline={3}").
        format(REGION, matchId, KEY, includeTimeline))

In [4]:
################## UNCOMMENT AND DELETE IRRELEVANT PARTS HERE ###############
################## DO NOT RUN THIS AGAIN #################

# List of starting MatchID's
# sean
match_id_list = [2029040788, 2028998508, 2028697992, 2028410883, 2028610390]
# jaemin
#match_id_list = [2028886202, 2028875395, 2028669368, 2028646565, 2028611382, 2026627159]
# lenny kei
#match_id_list = [2028144696, 2028076924, 2028017860, 2028129100, 2028115585, 2026610783]
# a paek
#match_id_list = [2028089458, 2027868106, 2027853644, 2027828374, 2026660053, 2026561470]

In [ ]:
# Constants
REGION = 'na'
STATUS_OK = 200
STATUS_RATE_LIMIT_EXCEEDED = 429
valid_rank_games = ['RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
                    'RANKED_PREMADE_3x3',  # Ranked Premade 3v3 games
                    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
                    'RANKED_TEAM_3x3',  # Ranked Team 3v3 games
                    'RANKED_TEAM_5x5']  # Ranked Team 5v5 games

# Starting MatchID
# STARTING_MATCH_ID = 2026937783

# Functions that check status of the data pulls
def isValid(match):
    if match.status_code != STATUS_OK:
        return False
    return True

def isClassicMatch(match):
    j = match.json()
    if j['mapId'] == 11 and j['matchMode'] == "CLASSIC" and j['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def isRankMatch(match):
    j = match.json()
    if j['queueType'] in valid_rank_games:
        return True
    else:
        return False
                    
def rateLimitExceeded(match):
    if int(match.status_code) == STATUS_RATE_LIMIT_EXCEEDED:
        return True
    return False

# index for the while-loop
i = 0
# counting the number of data retrieved
n = 0
# counting how many times we sleep in case we hit a dead-end
sleep_counter = 0

while i < 100000:
    # break out of the loop when there are no more match_id's left
    if match_id_list == []:
        print "ran out of starting match ID's. ending the loop"
        break
    
    # incrementing index
    i += 1
    #print ("i: %d, match_id: %d") % (i, match_id_list[0] + i)    
    
    current_match_id = match_id_list[0] + i
    m = get_match(REGION, current_match_id, True)
    
    if not rateLimitExceeded(m):
        if isValid(m) and isClassicMatch(m) and isRankMatch(m):
            
            ################## UNCOMMENT AND DELETE IRRELEVANT PARTS HERE ###############
            # if your name is sean, uncomment this
            f = open("dump/" + str(i + match_id_list[0])+".json", 'w+')
            # if you went to Kareoke Thursday night, uncomment this
            #f = open("dump_jaemin/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
            # if your name is Lenny Key, uncomment this
            #f = open("dump_kenny/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
            # if you're the dying president of Samsung, uncomment this
            #f = open("dump_andrew/" + str(i + STARTING_MATCH_ID)+".json", 'w+')
                     
            f.write(m.text.encode('utf-8'))
            n += 1
            print 'n: %d, ts: %s, gamdID: %d, date: %s, season: %s, tier: %s, loop_index: %d' % (n, datetime.datetime.fromtimestamp(time.time()).strftime('%m-%d %H:%M:%S'), 
                                                                                                 current_match_id, 
                                                                                                 ('%s' % (time.strftime('%m-%d', time.localtime(m.json()['matchCreation']/1000)))),
                                                                                                 m.json()['season'], m.json()['participants'][1]['highestAchievedSeasonTier'], i)
            f.close()
            # reset sleep_counter if we are get data again
            sleep_counter = 0
    else:
        sleep_counter += 1
        print("API rate limit reached - waiting 10 seconds, sleep count: %d, loop_index: %d") % (sleep_counter, i)
        time.sleep(10)
        
        # if we have to sleep 30 times, we will assume that we are in no man's land and move onto the next id on list
        if sleep_counter >= 30:
            del match_id_list[0]
            sleep_counter = 0
            i = 0
            print ("Seems like we hit a dead-end. Starting from the next match id on the list. new_starting_point: %d") % match_id_list[0]

n: 1, ts: 12-06 00:47:40, gamdID: 2029040789, date: 12-05, season: PRESEASON2016, tier: SILVER, loop_index: 1
n: 2, ts: 12-06 00:47:41, gamdID: 2029040791, date: 12-05, season: PRESEASON2016, tier: BRONZE, loop_index: 3
n: 3, ts: 12-06 00:47:42, gamdID: 2029040794, date: 12-05, season: PRESEASON2016, tier: UNRANKED, loop_index: 6
n: 4, ts: 12-06 00:47:43, gamdID: 2029040795, date: 12-05, season: PRESEASON2016, tier: GOLD, loop_index: 7
API rate limit reached - waiting 10 seconds, sleep count: 1, loop_index: 11
API rate limit reached - waiting 10 seconds, sleep count: 2, loop_index: 22
API rate limit reached - waiting 10 seconds, sleep count: 3, loop_index: 33
n: 5, ts: 12-06 00:48:32, gamdID: 2029040830, date: 12-05, season: PRESEASON2016, tier: SILVER, loop_index: 42
API rate limit reached - waiting 10 seconds, sleep count: 1, loop_index: 44
n: 6, ts: 12-06 00:48:48, gamdID: 2029040842, date: 12-05, season: PRESEASON2016, tier: SILVER, loop_index: 54
API rate limit reached - waiting 1